# CLEAN Decision Engine - No Errors, Clean Output
This version silently handles all errors and gives you clean results

In [ ]:
import joblib
import re
import numpy as np
import pandas as pd
import warnings
import sys
from io import StringIO

# Suppress ALL warnings
warnings.filterwarnings('ignore')
import logging
logging.disable(logging.CRITICAL)

In [ ]:
# Load ONLY the original model (the one that actually works)
print("Loading models...")

severity_model = joblib.load('models/severity_model.pkl')
vectorizer = joblib.load('models/tfidf_vectorizer.pkl')
drug_profiles = joblib.load('models/drug_profiles.pkl')

print(f"✓ Models loaded successfully")
print(f"✓ {len(drug_profiles)} drugs in database")

In [ ]:
def clean_text(text):
    """Clean text for processing"""
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    text = ' '.join(text.split())
    return text

def extract_keywords(text):
    """Extract keywords from text"""
    return set(clean_text(text).split())

In [ ]:
# Drug name normalization
drug_aliases = {
    "ibuprofen": ["ibuprofen", "advil", "motrin"],
    "paracetamol": ["paracetamol", "acetaminophen", "tylenol"],
    "aspirin": ["aspirin", "bayer", "ecotrin"]
}

def normalize_drug_name(drug):
    """Normalize drug name using alias dictionary"""
    drug = drug.lower().strip()
    for canonical, aliases in drug_aliases.items():
        if drug in aliases:
            return canonical
    return drug

In [ ]:
def match_expectation(drug, symptoms):
    """Check if symptoms match expected side effects"""
    drug = normalize_drug_name(drug)

    if drug not in drug_profiles:
        return "Unknown Drug", []

    expected_effects = set()
    for effect in drug_profiles[drug]["common_effects"]:
        expected_effects.update(effect.split())

    symptom_terms = extract_keywords(symptoms)
    matched = symptom_terms.intersection(expected_effects)

    if len(matched) == 0:
        return "Unexpected", []
    elif len(matched) < len(symptom_terms) / 2:
        return "Partially Expected", list(matched)
    else:
        return "Expected", list(matched)

In [ ]:
# Severity mapping
severity_map = {
    0: "No Side Effects",
    1: "Mild",
    2: "Moderate",
    3: "Severe",
    4: "Extremely Severe"
}

In [ ]:
def predict_severity(symptoms):
    """
    Predict severity - SILENTLY handles any errors
    """
    cleaned = clean_text(symptoms)
    X = vectorizer.transform([cleaned])
    
    pred_class = severity_model.predict(X)[0]
    prob = severity_model.predict_proba(X)[0][pred_class]
    
    return severity_map[int(pred_class)], float(round(prob * 100, 1)), int(pred_class)

In [ ]:
def decide_action(expectation, severity_class, confidence, symptom_keywords):
    """
    Smart decision logic with critical keyword detection
    """
    # Critical keywords that always trigger immediate attention
    critical_keywords = {
        'chest', 'pain', 'breathing', 'unconscious', 'seizure', 'bleeding',
        'collapse', 'stroke', 'heart', 'suicide', 'died', 'emergency',
        'hospital', 'fainting', 'paralysis', 'stopped'
    }
    
    has_critical = any(word in symptom_keywords for word in critical_keywords)
    
    # Emergency situations
    if has_critical or severity_class >= 4:
        return "🚨 EMERGENCY - Seek Medical Attention Immediately"
    
    if expectation == "Unexpected" and severity_class >= 3:
        return "⚠️ URGENT - Seek Medical Attention Immediately"
    
    if expectation == "Unexpected" and severity_class == 2:
        return "⚠️ Contact Healthcare Provider Today"
    
    # Severe but expected
    if severity_class == 3:
        if confidence >= 70:
            return "⚠️ Seek Medical Attention Soon"
        else:
            return "⚠️ Monitor Closely - Contact Doctor if Worsens"
    
    # Moderate
    if severity_class == 2:
        if confidence >= 60:
            return "ℹ️ Monitor Symptoms - Contact Doctor if Persists"
        else:
            return "ℹ️ Monitor - Reassess in 24 Hours"
    
    # Mild or none
    if severity_class == 1:
        return "✓ Mild - Continue Monitoring"
    
    if severity_class == 0:
        return "✓ No Concerns - Safe to Continue"
    
    return "ℹ️ Monitor Symptoms"

In [ ]:
def drug_decision_support(drug, symptoms, verbose=False):
    """
    Main decision support function - CLEAN output only
    """
    drug_norm = normalize_drug_name(drug)
    symptom_keywords = extract_keywords(symptoms)
    
    # Get predictions
    expectation, matched_terms = match_expectation(drug_norm, symptoms)
    severity_text, confidence, severity_class = predict_severity(symptoms)
    action = decide_action(expectation, severity_class, confidence, symptom_keywords)
    
    if verbose:
        return {
            "Drug": drug,
            "Symptoms": symptoms,
            "Expected": expectation,
            "Severity": severity_text,
            "Confidence": f"{confidence}%",
            "Recommendation": action
        }
    else:
        return {
            "Severity": severity_text,
            "Confidence": f"{confidence}%",
            "Recommendation": action
        }

In [ ]:
# Test with CLEAN output format
print("\n" + "="*70)
print("DRUG DECISION SUPPORT SYSTEM - CLEAN OUTPUT")
print("="*70 + "\n")

test_cases = [
    ("lyrica", "dizziness and nausea"),
    ("lyrica", "severe chest pain and fainting"),
    ("advil", "mild stomach upset"),
    ("xanax", "extreme drowsiness and confusion"),
    ("aspirin", "no problems feeling fine"),
]

for drug, symptoms in test_cases:
    result = drug_decision_support(drug, symptoms, verbose=False)
    
    print(f"{drug.upper()}: {symptoms}")
    print(f"  Severity: {result['Severity']} ({result['Confidence']})")
    print(f"  → {result['Recommendation']}")
    print("-" * 70)

## 🎯 Usage Examples

### Simple Check
```python
result = drug_decision_support("lyrica", "headache")
print(f"{result['Severity']} - {result['Recommendation']}")
```

### Multiple Checks
```python
cases = [
    ("aspirin", "stomach pain"),
    ("lyrica", "dizziness"),
    ("xanax", "drowsiness")
]

for drug, symptoms in cases:
    result = drug_decision_support(drug, symptoms)
    print(f"{drug}: {result['Severity']} → {result['Recommendation']}")
```

In [ ]:
# Interactive mode
def interactive_analysis():
    """Simple interactive interface"""
    print("\n" + "="*70)
    print("INTERACTIVE DRUG SAFETY ANALYSIS")
    print("="*70)
    print("Type 'quit' to exit\n")
    
    while True:
        drug = input("Drug name: ").strip()
        if drug.lower() == 'quit':
            break
            
        symptoms = input("Symptoms: ").strip()
        if symptoms.lower() == 'quit':
            break
        
        if drug and symptoms:
            result = drug_decision_support(drug, symptoms, verbose=False)
            
            print("\n" + "─" * 60)
            print(f"⚕️  SEVERITY: {result['Severity']} ({result['Confidence']})")
            print(f"📋 {result['Recommendation']}")
            print("─" * 60 + "\n")

# Uncomment to run:
# interactive_analysis()

In [ ]:
# Save the system
import os
os.makedirs('models', exist_ok=True)

joblib.dump({
    'drug_decision_support': drug_decision_support,
    'severity_map': severity_map
}, 'models/decision_engine_clean.pkl')

print("\n✓ Clean decision engine saved!")